In [65]:
VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7

In [64]:
SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}

In [34]:
def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open('words.txt', 'r')
    # wordList: list of strings
    wordList = []
    for line in inFile:
        wordList.append(line.strip().lower())
    print("  ", len(wordList), "words loaded.")
    return wordList

wordList = loadWords()

Loading word list from file...
   83667 words loaded.


In [52]:
def getWordScore(word, n):
    """
    Returns the score for a word. Assumes the word is a valid word.

    The score for a word is the sum of the points for letters in the
    word, multiplied by the length of the word, PLUS 50 points if all n
    letters are used on the f py irst turn.

    Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is
    worth 3, D is worth 2, E is worth 1, and so on (see SCRABBLE_LETTER_VALUES)

    word: string (lowercase letters)
    n: integer (HAND_SIZE; i.e., hand size required for additional points)
    returns: int >= 0
    """
    hold = [ SCRABBLE_LETTER_VALUES[i] for i in word ]
    score = sum(hold) * len(word)
    if len(hold) == n:
        score += 50
    return score

In [21]:
def getFrequencyDict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq

In [22]:
getFrequencyDict('hellou')

{'e': 1, 'h': 1, 'l': 2, 'o': 1, 'u': 1}

In [26]:
x = getFrequencyDict('hellou')

In [89]:
def displayHand(hand):
    """
    Displays the letters currently in the hand.

    For example:
    >>> displayHand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    for letter in hand.keys():
        for j in range(hand[letter]):
             print(letter,end=" ")       # print all on the same line
    print()                             # print an empty line

In [90]:
displayHand(x)

h e l l o u 


In [27]:
def updateHand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it. 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """
    copy = hand.copy()
    for i in word:
        if copy[i]:
            copy[i] -=1
    return copy

In [30]:
i = {'r': 2, 't': 2, 'p': 3, 'l': 2, 'a': 2, 'c': 2} 
updateHand(i,'claptrap')

{'a': 0, 'c': 1, 'l': 1, 'p': 1, 'r': 1, 't': 1}

In [31]:
i

{'a': 2, 'c': 2, 'l': 2, 'p': 3, 'r': 2, 't': 2}

In [41]:
def isValidWord(word, hand, wordList):
    """
    Returns True if word is in the wordList and is entirely
    composed of letters in the hand. Otherwise, returns False.

    Does not mutate hand or wordList.
   
    word: string
    hand: dictionary (string -> int)
    wordList: list of lowercase strings  \ # Problem #4: Playing a hand
    """
    handcopy = hand.copy()
    for i in word:
        if not handcopy[i] or handcopy[i] is 0:
            return False
        else:
            handcopy[i] -= 1
        
    if word in wordList:
        return True
    else:
        return False

In [42]:
isValidWord('rapture', {'r': 1, 'a': 3, 'p': 2, 'e': 1, 't': 1, 'u': 1}, wordList)

False

In [44]:
displayHand({'r': 1, 'a': 3, 'p': 2, 'e': 1, 't': 1, 'u': 1})

r a a a p p e t u 


In [45]:
r = {'r': 1, 'a': 3, 'p': 2, 'e': 1, 't': 1, 'u': 1}

In [46]:
sum([ x for x in r.values() ])

9

In [49]:
def calculateHandlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string int)
    returns: integer
    """
    return sum([ x for x in hand.values() ])

In [50]:
calculateHandlen(r)

9

In [81]:
def playHand(hand, wordList, n):
    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    * The user may input a word or a single period (the string ".") 
      to indicate they're done playing
    * Invalid words are rejected, and a message is displayed asking
      the user to choose another word until they enter a valid word or "."
    * When a valid word is entered, it uses up letters from the hand.
    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.
    * The sum of the word scores is displayed when the hand finishes.
    * The hand finishes when there are no more unused letters or the user
      inputs a "."

      hand: dictionary (string -> int)
      wordList: list of lowercase strings
      n: integer (HAND_SIZE; i.e., hand size required for additional points)
      
    """
    # Keep track of the total score
    hand_len = calculateHandlen(hand)
    score = 0
    # As long as there are still letters left in the hand:
    while hand_len > 0:
        # Display the hand
        print('Current Hand:')  
        displayHand(hand)
        # Ask user for input
        user_input = input('Enter word, or a "." to indicate that you are finished: ')
        # If the input is a single period:
        if user_input == '.':
            # End the game (break out of the loop)  
            break
        # Otherwise (the input is not a single period):   
        else:
            if isValidWord(user_input, hand, wordList) is False:
                print('Not valid \n')
            # If the word is not valid:   
                # Reject invalid word (print a message followed by a blank line)
            else:
                # Otherwise (the word is valid):
                # Tell the user how many points the word earned, and the updated total score, in one line followed by a blank line 
                score += getWordScore(user_input, n)
                print(getWordScore(user_input, n), '\n')
                # Update the hand 
                hand = updateHand(hand, user_input)
    # Game is over (user entered a '.' or ran out of letters), so tell user the total score
    print('Goodbye! Total score:', score, 'points')

In [82]:
playHand({'h':1, 'i':1, 'c':1, 'z':1, 'm':2, 'a':1}, wordList, 7)

Current Hand:
h i c z m m a 
Enter word, or a "." to indicate that you are finished: him
24 

Current Hand:
c z m a 
Enter word, or a "." to indicate that you are finished: cam
21 

Current Hand:
z 
Enter word, or a "." to indicate that you are finished: .
Goodbye! Total score: 45 points


In [92]:
def cat():
    print('miaou')

In [98]:
print('hellou', end=' ') 
cat()

hellou miaou


In [96]:
cat()

miaou
